In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Course_work/notebooks/data/df_mol2vec.csv')
df.drop(columns = ['Unnamed: 0'], inplace = True)
df.head()

In [ ]:
df.shape

In [ ]:
df.affinity.isna().sum()

0

## Train/Val/Test separation

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = df.affinity.apply(lambda x: 10*x)
x = df.drop(columns = ['pdbcode', 'affinity']).copy()

In [ ]:
x_train, x_rem, y_train, y_rem = train_test_split(x, y, train_size=0.7)
x_val, x_test, y_val, y_test = train_test_split(x_rem, y_rem, test_size=0.33)

In [ ]:
y_train.isna().sum()

0

## NN inplementation

In [ ]:
import tensorflow as tf

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu'),
    tf.keras.layers.SimpleRNN(32),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])
model.summary()

ValueError: ignored

In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(512, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_squared_error'])
NN_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               154112    
                                                                 
 dense_13 (Dense)            (None, 256)               131328    
                                                                 
 dense_14 (Dense)            (None, 256)               65792     
                                                                 
 dense_15 (Dense)            (None, 256)               65792     
                                                                 
 dense_16 (Dense)            (None, 128)               32896     
                                                                 
 dense_17 (Dense)            (None, 1)                 129       
                                                                 
Total params: 450,049
Trainable params: 450,049
Non-tr

In [ ]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
NN_model.fit(x_train, y_train, epochs=200, batch_size=32, validation_data = (x_val, y_val), callbacks=callbacks_list)

403/408 [============================>.] - ETA: 0s - loss: 15.5553 - mean_squared_error: 408.1685
Epoch 5: val_loss improved from 14.46795 to 14.30111, saving model to Weights-005--14.30111.hdf5
408/408 [==============================] - 3s 8ms/step - loss: 15.5475 - mean_squared_error: 407.9034 - val_loss: 14.3011 - val_mean_squared_error: 350.1126
Epoch 6/200
  1/408 [..............................] - ETA: 4s - loss: 14.8345 - mean_squared_error: 366.5947

KeyboardInterrupt: ignored